**Processamento da Trusted** - Desafio Parte III
- Carga histórica responsável pelo processamento dos arquivos CSV.

In [ ]:
!pip install boto3
!pip install pyspark

In [ ]:
import os
import boto3
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Trusted-CH").getOrCreate()

In [ ]:
aws_access_key_id = ''
aws_secret_access_key = ''
aws_session_token = ''

client = boto3.client('s3', aws_access_key_id = aws_access_key_id,
                  aws_secret_access_key = aws_secret_access_key,
                  aws_session_token = aws_session_token)

In [ ]:
bucket_name = "data-lake-livia"
input_csv_key = "Ram/Local/CSV/Movies/2024/03/04/movies.csv"

s3_object = client.get_object(Bucket=bucket_name, Key=input_csv_key)
df_pd = pd.read_csv(s3_object['Body'], sep="|", encoding="utf-8")
print(df_pd.head())

In [ ]:
df_pd.replace(r'\N', '', inplace=True)

In [ ]:
df_csv = spark.createDataFrame(df_pd)
df_csv.printSchema()

In [ ]:
from pyspark.sql.types import StringType, ArrayType
from pyspark.sql.functions import col, split
df = df_csv\
    .withColumn("genero", split(col("genero"), ",").cast(ArrayType(StringType())))\
    .withColumn("profissao", split(col("profissao"), ",").cast(ArrayType(StringType())))\
    .withColumn("titulosMaisConhecidos", split(col("titulosMaisConhecidos"), ",").cast(ArrayType(StringType())))

In [ ]:
df.show(5)

In [ ]:
file_name = "movies.parquet"

file_movies = f'/content/{file_name}'
df.coalesce(1).write.parquet(file_movies, mode="overwrite")

for filename in os.listdir(file_movies):
    if filename.startswith('part-'):
        new_filename = file_name
        old_filepath = os.path.join(file_movies, filename)
        new_filepath = os.path.join(file_movies, new_filename)
        os.rename(old_filepath, new_filepath)

In [ ]:
folder_name = "Movies"

up_file_movies = f'/content/{file_name}/{file_name}'
aws_file_movies = f"TRT/{folder_name}/{file_name}"

try:
    client.upload_file(up_file_movies, bucket_name, aws_file_movies)
    print(f"Sucesso! Arquivo parquet enviado para o S3.")
except Exception as e:
    print(f"Erro durante o upload: {e}")